In [ ]:
import wikichatter as wc
import mwapi
import requests
import json
import csv

In [ ]:
#When working with mwapi, you should begin your file with a session, with a user_agent
#this allows you to make multiple calls without getting shut out of the api

session = mwapi.Session('https://en.wikipedia.org', user_agent='ewhit')

In [ ]:
#params holds the parameters you want to use when calling mwapi. This makes it easy to 
#manipulate each parameter, particularly "titles"
#Here, we can see that titles has been set to "Wikipedia:List of controversial issues"
#These parameters will do the following:
    #Tell mwapi to make a query (action), to get all of the links present (prop) on the page 
    #"Wikipedia:List of controversial issues" (titles), and to return this in json format (format)

def params():
    return{
    "action": "query",
    "titles" : "Wikipedia:List of controversial issues",
    "prop": "links",
    "format": "json",
    "pllimit" : "1"
    }


#kwargs calls params in order to be set - this is what you will end up passing
kwargs=params()

#This holds the results of your query, and continuation=True tells session.get to keep going when it reaches the
#limit (of 50, I believe), so that you can get *all* of the links on the page
#Because we wanted all the titles on the page, query will be a generator
query = session.get(**kwargs, continuation = True)

In [ ]:
#Pagelist is a list that will ultimately hold the titles of all the controversial pages
pageList = []

#This loops through all the results of the earlier query
#However, because what we want to do next is acquire the talk page for each controversial
#page, we will need to do a bit of trimming
#Ultimately, this bit of code will put "Talk:page title" into our list of pages to visit later
for request in query:
    title = json.dumps(request)
    index = title.find('links')
    temp = title[index:]
    index2 = temp.find('title')
    temp2 = temp[index2:]
    temp3 = temp2[9:]
    index3 = temp3.find('"')
    temp4 = temp3[:index3]
    toTalk = "Talk:"
    temp5 = toTalk + temp4
    pageList.append(temp5)
    

print("end for loop")

In [ ]:
#This is a dictionary to hold the talk page title, and the content of that talk page
content = {}

#This loops through each page in our list of controversial talk pages
for page in pageList:
    
    #Once again, we want to define the parameters we will send to session.get
    #Here, we are again querying, and "titles" : page will query the page with the corresponding title
    #(which is why we just stored the talk page titles earlier)
    #"prop" : "revisions" gets the most recent revision, and "rvprop" : "content" gets the current content
    def param(page):
        return{
        "action": "query",
        "titles" : page,
        "prop" : "revisions",
        "rvprop" : "content",
        "format": "json",
        }
    
    #Here we again set kwargs to send to session.get
    kwargs2 = param(page)
    query = session.get(**kwargs2, continuation=True )
    
    #Because this portion of code was throwing errors, I have used several try-catch blocks
        #Several were used because the first kind returned a TypeError, so I caught that as well
    #try-catch block
    for request in query:
        #This is the text that results from the individual query (so, the talk page content)
        text = json.dumps(request)

        try:
            #Here, we try to parse the talk page content
            try:
                parsed_text = wc.parse(text)
            except NameError:
                print("error")
                print("\n")
            except NoUsernameError(parsed_text):
                print("error")
                print("\n")
        except TypeError:
            print("error")
            print("\n")

        #Here, we add an entry to the dictionary of the current page's title and parsed text
        content[page] = parsed_text
            
print("finished querrying")

In [14]:
#This simply prints the page titles and their corresponding cleaned text to a file
with open("parsedTalkPages.csv", "w", newline = '') as w:
    writer = csv.writer(w, delimiter = ',')
    for key, value in content.items():
        writer.writerow([key, value])

end for loop
error #:%s 1


{"batchcomplete": "", "query": {"pages": {"1915271": {"pageid": 1915271, "title": "Talk:1919 World Series", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{Talk header}}\n{{WikiProject banner shell|1=\n{{WikiProject United States|class=C|importance=Low|OH=yes|OH-importance=Mid}}\n{{WikiProject Illinois|class=C  |importance=Low}}\n{{WikiProject Chicago|class=C |importance=Mid }}\n{{WikiProject Baseball|reds=yes|importance=high |class=C }}}}\n{{On this day|date1=2005-10-09|oldid1=25124080|date2=2006-10-09|oldid2=80424092}}\n\n== Black Sox Scandal == thank you for helping me with my history project Wikipedia!\nWho is Mike Jones?? No one has ever claimed Kid Gleason (the White Sox manager) was in on the conspiracy. The first two sentences of this entry read like someone's private goof. Chick Gandil, the first baseman of the White Sox and not the manager, is widely believed to have engineered the conspiracy on the players' side.

error #:%s 24


{"batchcomplete": "", "query": {"pages": {"6492193": {"pageid": 6492193, "title": "Talk:Adventism", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{talk header}}\n{{WikiProject Christianity|class=Start|importance=Top|seventh-day-adventist-church=yes|seventh-day-adventist-church-importance=Top|core-topics-work-group = yes|core-topics-work-group-importance=low}}\n{{WikiProject Theology|class=Start|importance=Mid}}\n\n==Start==\nI'm an Adventist. Let's build us a page.\n\nWhat would you like to see on an Adventist Page? \nI would like a page on Adventist Beliefs a Forum for people to speak about how Adventists needs to change to be more relevant in today\u2019s world. MPP\n\n:What we shall see in an article about ''Adventist'' is determined by the purpose and policies of Wikipedia. This is an [[#Encyclopedia|encyclopedia]], I'll answer more [[#Encyclopedia|below]]. <span style=\"color: #800000; background-color: #FFFFA0; padding: 1px 2px 

error #:%s 69


{"batchcomplete": "", "query": {"pages": {"1442481": {"pageid": 1442481, "title": "Talk:Andrew Dice Clay", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{WikiProject Biography\n|living=yes\n|class=Start\n|filmbio-work-group=yes\n|listas=Clay, Andrew Dice\n}}\n{{WikiProject Comedy|class=start|importance=low}}\n\n== Lifetime ban ==\nI put quotation marks on \"lifetime ban\" because I clearly recall seeing Dice on [[MTV]] in at least one subsequent appearance with David Spade. Spade played his trademark receptionist character who refused to allow Dice into the [[MTV Video Music Awards]], citing the \"lifetime ban.\" However, by appearing on the network the \"lifetime ban\" was clearly not taken very seriously. I suspect he has appeared in other segments but that's the only one I've ever seen. --[[User:Feitclub|Feitclub]] 22:11, Jan 28, 2005 (UTC)\n\n:There's also nothing but a parting mention in here about the incident in question, and t

error #:%s 162


{"batchcomplete": "", "query": {"pages": {"1717998": {"pageid": 1717998, "title": "Talk:Black hole information paradox", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{WikiProject Physics|class=start|importance=mid|relativity=yes}}\n\n== Frequently Asked Question ==\nA '''frequently asked question''' on this talk page is:\n<blockquote>''Is it really true that in Physics is it presumed that \"information cannot be destroyed\" ?''</blockquote>\n\nNo. The laws of thermodynamic state the opposite. That a complicated organized object like a book, containing a large amount of information can be scrambled irreversibly by burning it until all information in it is lost. It will go up in smoke. Quantum indeterminacy (fuzziness) adds to the irretrievable nature of the lost information. There is no scientific consensus that a \"Law Of Conservation of Information\" has been proved.   <small class=\"autosigned\">\u2014&nbsp;Preceding [[Wikipedia:S

error #:%s 165


{"batchcomplete": "", "query": {"pages": {"21090455": {"pageid": 21090455, "title": "Talk:Blood type", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{Talk header}}\n{{Article history|action1=GAN\n|action1date=20:25, 28 December 2006\n|action1result=listed\n|action1link=Talk:Blood type#GA review\n|action1oldid=96992497\n\n|action2=GAR\n|action2date=July 3, 2009\n|action2result=Delisted\n|action2link=Talk:Blood type/GA1\n|action2oldid=299759891\n\n|currentstatus=DGA\n|topic=Natsci\n}}\n{{Vital article|level=4|topic=Science|class=B|subpage=Biology}}\n{{WikiProjectBannerShell|1=\n{{WikiProject Genetics|class=b|importance=high}}\n{{WikiProject Medicine|class=B|importance=High|MCOTW=prev|selected=yes|hemonc=yes |hemonc-imp= }}\n{{WP1.0|WPCD=yes|class=b|importance=top}}\n{{WikiProject Physiology|class=B|importance=high|field=blood}}}}\n{{To do}} \nThere exists a z negative blood type\n\n== The Heredity of blood ==\n\nI wonder whether the au

...

error #:%s 916


{"batchcomplete": "", "query": {"pages": {"1387835": {"pageid": 1387835, "title": "Talk:Panama Canal Zone", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{WikiProjectBannerShell|1=\n{{WikiProject Panama|class=B|importance=Top}}\n{{WikiProject United States|class=B|importance=Mid}}\n{{WikiProject Latin America|class=B|importance=low}}\n{{WikiProject Former countries\n|class=C\n|attention= \n|structureneeded=\n|flagneeded=No\n|coatneeded=No\n|mapneeded=No\n|infoboxneeded=No\n|B-Class-1= <!-- 1. It is suitably referenced, and all major points are appropriately cited. -->\n|B-Class-2= <!-- 2. It reasonably covers the topic, and does not contain major omissions or inaccuracies. -->\n|B-Class-3= <!-- 3. It has a defined structure, including a lead section and one or more sections of content. -->\n|B-Class-4= <!-- 4. It is free from major grammatical errors. -->\n|B-Class-5= <!-- 5. It contains appropriate supporting materials, such as an i

error #:%s 923


{"batchcomplete": "", "query": {"pages": {"20903973": {"pageid": 20903973, "title": "Talk:Patriot Act", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{Talk header}}\n\n\n{{ArticleHistory|action1=GAN\n|action1date=30 September 2006\n|action1link=Talk:USA PATRIOT Act/Archive 1#GA Nomination\n|action1result=Failed\n|action1oldid=78545730\n\n|action2=PR\n|action2date=15 October 2006\n|action2link=Wikipedia:Peer review/USA PATRIOT Act/archive1\n|action2result=Reviewed\n|action2oldid=164680105\n\n|action3=FAC\n|action3date=15:20, 8 September 2008\n|action3link=Wikipedia:Featured article candidates/USA PATRIOT Act/archive1\n|action3result=not promoted\n|action3oldid=236995526\n\n|currentstatus=FFAC\n}}\n{{WikiProjectBannerShell|1=\n{{WikiProject United States|class=B|importance=Mid|USGov=yes|USGov-importance=mid}}\n{{WikiProject Law|class=B|importance=Mid}}\n{{WikiProject Mass surveillance|class=B|importance=High}}\n{{WikiProject Freedom of

error #:%s 1138


{"batchcomplete": "", "query": {"pages": {"7066060": {"pageid": 7066060, "title": "Talk:Smart meter", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{WikiProject Energy|class=Start|importance=low}}\n{{WikiProject Technology|class=c|importance=low}}\n{{Energy portal news|date={{date|2008-10-28}}}}\n{{not a forum|[[smart meter]]s}}\n{{controversial}}\n\n== Google Power is discontinued ==\nGoogle power is now discontinued. Would recommend adding to page where it is mentioned  <span style=\"font-size: smaller;\" class=\"autosigned\">\u2014 Preceding [[Wikipedia:Signatures|unsigned]] comment added by [[Special:Contributions/218.215.172.113|218.215.172.113]] ([[User talk:218.215.172.113|talk]]) 12:42, 6 August 2011 (UTC)</span><!-- Template:Unsigned IP --> <!--Autosigned by SineBot-->\n== Not an article, but a biased screed ==\n\nReally one of the most biased and unfounded entries I've seen on Wikipedia.  This is not ABOUT smart meters, it

error #:%s 1177


{"batchcomplete": "", "query": {"pages": {"16830491": {"pageid": 16830491, "title": "Talk:Sting", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{WikiProject Disambiguation}}\n\n==Untitled==\nSeriously, Steve Borden owns the trademark - he should be listed first. [[User:Ascensionblade|Ascensionblade]] ([[User talk:Ascensionblade|talk]]) 19:26, 2 September 2008 (UTC)\n:The ordering should be done in accordance with the [[Wikipedia:Manual of Style (disambiguation pages)#Order_of_entries|Wikipedia:Manual of Style (disambiguation pages)]] which states:\n\n::In most cases, place the items in order of usage, with the most-used meanings appearing at the top and less common meanings below.\n\n:::Steve Borden owns A trademark, not THE trademark. More specifically he registered 'Sting' for use in entertainment services, namely live and televised performances by a professional wrestler/entertainer. This falacy that he owns THE trademark 'Sting'

finished querrying
